# 🚨 The Problem: Progress Bars Break GitHub Notebook Rendering

When you run long tasks in notebooks—like downloading large models or datasets—you often see progress bars that keep you informed. Popular libraries like Hugging Face's Transformers use **tqdm-based progress bars** for this.

However, these progress bars create **interactive widget metadata inside the notebook's JSON**, which GitHub tries to render. If this metadata is incomplete or corrupted, GitHub throws errors such as:

> Invalid Notebook  
> There was an error rendering your Notebook: the 'state' key is missing from 'metadata.widgets'. Add 'state' to each, or remove 'metadata.widgets'.  

This means GitHub cannot display your notebook preview properly as shown below.

![Invalid Notebook Error on GitHub](https://raw.githubusercontent.com/dimitrisdais/generative-ai-lab/main/assets/images/invalid_notebook_error.png)


# ❓ Why This Happens

The widget metadata (in the notebook JSON) contains information needed to render interactive elements. When you run progress bars or ipywidgets, this metadata is created. If you manually edit or upload a notebook with **incomplete or broken widget metadata**, GitHub refuses to render it.

One common example is loading a Hugging Face summarization model in Google Colab:

![Hugging Face Progress Bars causing error](https://raw.githubusercontent.com/dimitrisdais/generative-ai-lab/main/assets/images/progress_bars_error.png)

# 🛠️ Clean Widget Metadata Programmatically After Running

You can keep your outputs but remove the problematic widget metadata using a cleanup script.

The following script loads your notebook JSON, deletes all "widgets" entries from metadata (both cell-level and notebook-level), and saves a cleaned notebook.

In [ ]:
import json
from google.colab import drive

# Step 1: Mount your Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Step 2: Define the full path to your notebook file in Drive
notebook_path = '/content/drive/MyDrive/Colab Notebooks/0003_widget_cleanup_for_github_upload/dummy_notebook.ipynb'
cleaned_path = '/content/drive/MyDrive/Colab Notebooks/0003_widget_cleanup_for_github_upload/dummy_notebook_cleaned.ipynb'

# Step 3: Load notebook JSON
with open(notebook_path, 'r', encoding='utf-8') as f:
    notebook = json.load(f)

# Step 3.1: Remove 'widgets' metadata from notebook-level metadata if present
if 'widgets' in notebook.get('metadata', {}):
    print("Removing top-level widgets metadata")
    del notebook['metadata']['widgets']

# Step 4: Remove 'widgets' metadata in all cells if present
for i, cell in enumerate(notebook.get('cells', [])):
    metadata = cell.get('metadata', {})
    if 'widgets' in metadata:
        print(f"Removing widgets metadata in cell {i}")
        del metadata['widgets']

# Step 5: Save cleaned notebook to a new file in Drive or local
with open(cleaned_path, 'w', encoding='utf-8') as f:
    json.dump(notebook, f, indent=2)

print(f"Cleaned notebook saved at: {cleaned_path}")


This removes progress bars but keeps textual output and warnings.

![Notebook after widget cleanup (bars removed) - Google Colab view](https://raw.githubusercontent.com/dimitrisdais/generative-ai-lab/main/assets/images/cleaned_notebook.png)

![Notebook after widget cleanup (bars removed) - GitHub rendered view](https://raw.githubusercontent.com/dimitrisdais/generative-ai-lab/main/assets/images/cleaned_notebook_github.png)
